In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import zip_longest

In [2]:
data = json.load(open("2025-06-15_11-39-04-387039_measure_weight_FORCE_MEASUREMENT_CASE.json", encoding="utf-8"))

df = pd.json_normalize(
    data,
    record_path="raw_measurements",
    meta=["version","type","start_time","end_time","sampling_rate","weight_filter","zero_offset","expected_weight","robot_type","eoat_params"],
    errors="ignore"
)

print(df.shape)
print(df.keys())
print(df.head())

(493, 19)
Index(['timestamp', 'value', 'force_vector', 'robot_tcp.timestamp',
       'robot_tcp.flange', 'robot_tcp.tcp_offset', 'robot_tcp.velocity_linear',
       'robot_tcp.velocity_angular', 'robot_tcp.joint_angles', 'version',
       'type', 'start_time', 'end_time', 'sampling_rate', 'weight_filter',
       'zero_offset', 'expected_weight', 'robot_type', 'eoat_params'],
      dtype='object')
      timestamp  value          force_vector  robot_tcp.timestamp  \
0  1.749980e+09 -0.054   [0.603, 0.618, 9.0]         1.749980e+09   
1  1.749980e+09 -0.055    [0.647, 0.59, 9.0]         1.749980e+09   
2  1.749980e+09 -0.055   [0.683, 0.56, 10.0]         1.749980e+09   
3  1.749980e+09 -0.055  [0.711, 0.529, 11.0]         1.749980e+09   
4  1.749980e+09 -0.055   [0.73, 0.502, 11.0]         1.749980e+09   

                                    robot_tcp.flange  \
0  [[-0.5583840012550354, -0.3735954165458679, -0...   
1  [[-0.5583451986312866, -0.37359368801116943, -...   
2  [[-0.558345198

In [3]:
# Renaming and ordering the data.

df[["Mx","My","Fz"]] = pd.DataFrame(df.pop("force_vector").tolist(), index=df.index)

flange = pd.DataFrame(df.pop("robot_tcp.flange").tolist(), index=df.index)
df[["flange_tx","flange_ty","flange_tz"]] = pd.DataFrame(flange[0].tolist(), index=df.index)
df[["flange_qw","flange_qx","flange_qy","flange_qz"]] = pd.DataFrame(flange[1].tolist(), index=df.index)

tcp_off = pd.DataFrame(df.pop("robot_tcp.tcp_offset").tolist(), index=df.index)
df[["tcp_tx","tcp_ty","tcp_tz"]] = pd.DataFrame(tcp_off[0].tolist(), index=df.index)
df[["tcp_qw","tcp_qx","tcp_qy","tcp_qz"]] = pd.DataFrame(tcp_off[1].tolist(), index=df.index)

df[["tcp_vx","tcp_vy","tcp_vz"]] = pd.DataFrame(df.pop("robot_tcp.velocity_linear").tolist(), index=df.index)

df[["tcp_wx","tcp_wy","tcp_wz"]] = pd.DataFrame(df.pop("robot_tcp.velocity_angular").tolist(), index=df.index)

df[["joint_base","joint_shoulder","joint_elbow","joint_wrist_1","joint_wrist_2","joint_wrist_3"]] = (
    pd.DataFrame(df.pop("robot_tcp.joint_angles").tolist(), index=df.index)
)


t_num = pd.to_numeric(df["timestamp"], errors="coerce")
t_rel_ms_num = (t_num - t_num.iloc[0]) * 1000
t_dt = pd.to_datetime(df["timestamp"], errors="coerce")
t_rel_ms_dt = (t_dt - t_dt.iloc[0]) / pd.to_timedelta(1, "ms")
time_ms = t_rel_ms_num.fillna(t_rel_ms_dt)


rt_num = pd.to_numeric(df["robot_tcp.timestamp"], errors="coerce")
rt_rel_ms = (rt_num - rt_num.iloc[0]) * 1000

In [ ]:
plt.figure(figsize=(18, 20))

# 1) Weight & Fz (N) together
plt.subplot(5, 2, 1)
plt.plot(time_ms, df["value"], label="weight_kg")
plt.plot(time_ms, df["Fz"], label="Fz_N")
plt.xlabel("Time (ms)")
plt.ylabel("Value")
plt.title("Weight (kg) & Fz (N) over Time")
plt.legend()
plt.grid(True)

# 2) Moments Mx, My (Nm)
plt.subplot(5, 2, 2)
plt.plot(time_ms, df["Mx"], label="Mx_Nm")
plt.plot(time_ms, df["My"], label="My_Nm")
plt.xlabel("Time (ms)")
plt.ylabel("Moment (Nm)")
plt.title("Moments over Time")
plt.legend()
plt.grid(True)

# 3) TCP linear velocity (m/s)
plt.subplot(5, 2, 3)
plt.plot(time_ms, df["tcp_vx"], label="tcp_vx")
plt.plot(time_ms, df["tcp_vy"], label="tcp_vy")
plt.plot(time_ms, df["tcp_vz"], label="tcp_vz")
plt.xlabel("Time (ms)")
plt.ylabel("Velocity (m/s)")
plt.title("TCP Linear Velocity")
plt.legend()
plt.grid(True)

# 4) TCP angular velocity (rad/s)
plt.subplot(5, 2, 4)
plt.plot(time_ms, df["tcp_wx"], label="tcp_wx")
plt.plot(time_ms, df["tcp_wy"], label="tcp_wy")
plt.plot(time_ms, df["tcp_wz"], label="tcp_wz")
plt.xlabel("Time (ms)")
plt.ylabel("Angular Velocity (rad/s)")
plt.title("TCP Angular Velocity")
plt.legend()
plt.grid(True)

# 5) Joint angles (rad)
plt.subplot(5, 2, 5)
for c in ["joint_base","joint_shoulder","joint_elbow","joint_wrist_1","joint_wrist_2","joint_wrist_3"]:
    plt.plot(time_ms, df[c], label=c)
plt.xlabel("Time (ms)")
plt.ylabel("Angle (rad)")
plt.title("Joint Angles")
plt.legend()
plt.grid(True)
        
# 6) Flange translation (m)
plt.subplot(5, 2, 6)
plt.plot(time_ms, df["flange_tx"], label="flange_tx")
plt.plot(time_ms, df["flange_ty"], label="flange_ty")
plt.plot(time_ms, df["flange_tz"], label="flange_tz")
plt.xlabel("Time (ms)")
plt.ylabel("Translation (m)")
plt.title("Flange Translation")
plt.legend()
plt.grid(True)

# 7) Flange quaternion (unitless)
plt.subplot(5, 2, 7)
for c in ["flange_qw","flange_qx","flange_qy","flange_qz"]:
    plt.plot(time_ms, df[c], label=c)
plt.xlabel("Time (ms)")
plt.ylabel("Quaternion")
plt.title("Flange Quaternion")
plt.legend()
plt.grid(True)

# 8) TCP offset translation (m)
plt.subplot(5, 2, 8)
plt.plot(time_ms, df["tcp_tx"], label="tcp_tx")
plt.plot(time_ms, df["tcp_ty"], label="tcp_ty")
plt.plot(time_ms, df["tcp_tz"], label="tcp_tz")
plt.xlabel("Time (ms)")
plt.ylabel("Translation (m)")
plt.title("TCP Offset Translation")
plt.legend()
plt.grid(True)

# 9) TCP offset quaternion (unitless)
plt.subplot(5, 2, 9)
for c in ["tcp_qw","tcp_qx","tcp_qy","tcp_qz"]:
    plt.plot(time_ms, df[c], label=c)
plt.xlabel("Time (ms)")
plt.ylabel("Quaternion")
plt.title("TCP Offset Quaternion")
plt.legend()
plt.grid(True)

# 10) Robot TCP timestamp vs weight timestamp (ms)
plt.subplot(5, 2, 10)
plt.plot(time_ms, time_ms, label="weight_ts_rel_ms")
plt.plot(time_ms, rt_rel_ms, label="robot_tcp_ts_rel_ms")
plt.xlabel("Time (ms)")
plt.ylabel("Relative Time (ms)")
plt.title("Timestamp Alignment")
plt.legend()
plt.grid(True)


plt.tight_layout()
plt.show()

In [ ]:
#Export DataFrame

df.to_csv("mediciones_robot.csv", index=True)
